## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Load the prices on zero coupon bonds with maturities of 1 through 5 years.
data = pd.read_excel('data/famabliss_strips_2025-11-28.xlsx', sheet_name= 0)

# Convert date to datetime and set as index
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# Display the first few rows of the data
data.head()

,1,2,3,4,5
date,,,,,
1952-06-30,0.981778,0.962531,0.942467,0.926341,0.904183
1952-07-31,0.980948,0.960315,0.939312,0.923089,0.898516
1952-08-29,0.980578,0.959750,0.937819,0.921458,0.895717
1952-09-30,0.980312,0.959098,0.937551,0.920961,0.891676
1952-10-31,0.981108,0.959745,0.937583,0.919290,0.897248


## Data Preprocessing

In [3]:
# Select the last available trading day in November for each year from 2020 to 2025
november_dates = [
    data[(data.index.year == year) & (data.index.month == 11)].index[-1]
    for year in range(2020, 2026)
    if any((data.index.year == year) & (data.index.month == 11))
]

# Subset the data for these specific dates and ensure column names are integers
df_sim = data.loc[november_dates].copy()
df_sim.columns = df_sim.columns.astype(int)

df_sim

,1,2,3,4,5
date,,,,,
2020-11-30,0.998839,0.997047,0.994464,0.988809,0.981515
2021-11-30,0.997523,0.988614,0.974796,0.958322,0.943772
2022-11-30,0.954375,0.918220,0.887608,0.857356,0.830593
2023-11-30,0.951080,0.911951,0.876957,0.842191,0.809378
2024-11-29,0.959187,0.920923,0.885440,0.850418,0.817795
2025-11-28,0.964936,0.932866,0.900919,0.868240,0.836057


## 1.1 The Carry Trade

In [4]:
# Parameters
initial_mv = 100_000_000            # $100 million notional
haircut = 0.02                      # 2% haircut
capital = initial_mv * haircut * 2  # $4M total capital (for long + short)

# Initial Long Position
start_date = df_sim.index[0]
price_5y_start = df_sim.loc[start_date, 5]
long_qty = initial_mv / price_5y_start  # Face value of 5yr bond bought

# Setup Results DataFrame
results = pd.DataFrame(index=df_sim.index, columns=['Long_MV', 'Short_MV', 'Cash_Balance', 'Total_Wealth'])
results['Cash_Balance'] = 0.0

results.loc[start_date, 'Long_MV'] = initial_mv
results.loc[start_date, 'Short_MV'] = -initial_mv
results.loc[start_date, 'Total_Wealth'] = 0  # Start with net wealth zero (excluding capital)

current_short_qty = 0  # Track face value of the current short (negative = short)

# Simulate Year by Year
for i, date in enumerate(df_sim.index):
    prev_date = df_sim.index[i-1] if i > 0 else None
    years_rem = 5 - i

    # -- Update Long Position --
    if years_rem > 0:
        price_long = df_sim.loc[date, years_rem]
        mv_long = long_qty * price_long
    else:
        mv_long = long_qty * 1.0  # At maturity, receive par value

    results.loc[date, 'Long_MV'] = mv_long

    # -- Handle Short Side (Open, Close, Roll) --
    if i == 0:
        # Open initial short in 1-year bond; proceeds net out long purchase
        price_1y = df_sim.loc[date, 1]
        current_short_qty = -(initial_mv / price_1y)
        results.loc[date, 'Short_MV'] = -initial_mv
        results.loc[date, 'Cash_Balance'] = 0.0
    else:
        # Carry forward cash balance
        cash = results.loc[prev_date, 'Cash_Balance']

        # Close matured short (pay face value)
        cash += current_short_qty * 1.0  # Adds a negative amount (cash outflow at maturity)

        # Open new short in years 1–3 only (rolls into new 1yr short, three times)
        if i < 4:
            price_1y = df_sim.loc[date, 1]
            cash += initial_mv         # Receive proceeds from new short
            current_short_qty = -(initial_mv / price_1y)
            results.loc[date, 'Short_MV'] = -initial_mv
        else:
            # Last two years: no new short is opened; short position = 0
            current_short_qty = 0
            results.loc[date, 'Short_MV'] = 0

        results.loc[date, 'Cash_Balance'] = cash

    # -- Calculate Total Wealth --
    results.loc[date, 'Total_Wealth'] = (
        results.loc[date, 'Cash_Balance']
        + results.loc[date, 'Long_MV']
        + results.loc[date, 'Short_MV']
    )

# Final Metrics and Output
total_profit = results.loc[df_sim.index[-1], 'Total_Wealth']
total_return = total_profit / capital


display(results[['Long_MV', 'Short_MV', 'Cash_Balance', 'Total_Wealth']])
print(f"Total Profit (Nov 2025): ${total_profit:,.2f}")
print(f"Total Return on $4M Capital: {total_return:.2%}")

,Long_MV,Short_MV,Cash_Balance,Total_Wealth
date,,,,
2020-11-30,100000000.0,-100000000,0.000000e+00,0.0
2021-11-30,97637027.120245,-100000000,-1.162275e+05,-2479200.387266
2022-11-30,90432371.847526,-100000000,-3.645748e+05,-9932202.964478
2023-11-30,92912546.487828,-100000000,-5.145191e+06,-12232644.465758
2024-11-29,97725112.731931,0,-1.102888e+08,-12563720.439888
2025-11-28,101883281.158144,0,-1.102888e+08,-8405552.013675


Total Profit (Nov 2025): $-8,405,552.01
Total Return on $4M Capital: -210.14%


## 1.2

In [5]:
# Construct implied forward price table using Nov 2020 yield curve
# Forward price F(0, T1, T2) = P(0, T2) / P(0, T1)

df_fwd = df_sim.copy()
base_date = df_sim.index[0]
base_prices = df_sim.loc[base_date]

# Calculate hypothetical forward prices for each year after 2020
for i in range(1, len(df_sim)):
    current_date = df_sim.index[i]
    years_passed = i
    for duration in range(1, 6):
        maturity = years_passed + duration
        if maturity <= 5:
            df_fwd.loc[current_date, duration] = base_prices[maturity] / base_prices[years_passed]
        else:
            df_fwd.loc[current_date, duration] = np.nan  # Forward price undefined beyond available maturities

# --- Run carry strategy simulation using forward prices ---
results_fwd = pd.DataFrame(index=df_fwd.index, columns=['Long_MV', 'Short_MV', 'Cash_Balance', 'Total_Wealth'])
results_fwd['Cash_Balance'] = 0.0

# Initial long and short positions (Nov 2020)
price_5y_start = df_fwd.loc[start_date, 5]
long_qty = initial_mv / price_5y_start
results_fwd.loc[start_date, 'Long_MV'] = initial_mv
results_fwd.loc[start_date, 'Short_MV'] = -initial_mv
results_fwd.loc[start_date, 'Total_Wealth'] = 0
current_short_qty = 0

for i, date in enumerate(df_fwd.index):
    prev_date = df_fwd.index[i-1] if i > 0 else None
    # Long position: value as 5-year bond amortizes
    years_remaining = 5 - i
    if years_remaining > 0:
        mv_long = long_qty * df_fwd.loc[date, years_remaining]
    else:
        mv_long = long_qty * 1.0  # Redeems at par at maturity
    results_fwd.loc[date, 'Long_MV'] = mv_long

    # Short position + cash management
    if i == 0:
        price_1y = df_fwd.loc[date, 1]
        current_short_qty = -initial_mv / price_1y
        results_fwd.loc[date, 'Short_MV'] = -initial_mv
    else:
        cash = results_fwd.loc[prev_date, 'Cash_Balance']
        # Settle matured short (pay face value)
        cash += current_short_qty * 1.0
        if i < 4:
            price_1y = df_fwd.loc[date, 1]
            cash += initial_mv  # Proceeds from new 1y short sale
            current_short_qty = -initial_mv / price_1y
            results_fwd.loc[date, 'Short_MV'] = -initial_mv
        else:
            current_short_qty = 0
            results_fwd.loc[date, 'Short_MV'] = 0
        results_fwd.loc[date, 'Cash_Balance'] = cash

    # Total portfolio value
    results_fwd.loc[date, 'Total_Wealth'] = (
        results_fwd.loc[date, 'Cash_Balance']
        + results_fwd.loc[date, 'Long_MV']
        + results_fwd.loc[date, 'Short_MV']
    )

total_profit_fwd = results_fwd.loc[df_fwd.index[-1], 'Total_Wealth']
total_return_fwd = total_profit_fwd / capital

print(f"Total Profit (Nov 2025): ${total_profit_fwd:,.2f}")
print(f"Total Return on Capital: {total_return_fwd:.2%}")

Total Profit (Nov 2025): $755,678.03
Total Return on Capital: 18.89%


## 1.3


**Less Favorable**

**Reasoning:**

* **Fact 3 of the EH:** Dynamic (conditional) tests of the Expectations Hypothesis (such as Fama-Bliss regressions) demonstrate that **forward rates are biased upward predictors** of future spot rates. This bias represents a **term premium** that is time-varying and highly correlated with the **slope of the yield curve**. <br><br>
* **Context of Nov 2020:** The yield curve was exceptionally **steep** (short rates near 0.1%, long rates significantly higher). Per Fact 3, a steep slope predicts **high expected excess returns** for long bonds. Ex-ante, the trade looked highly favorable because the term premium being "harvested" was at a cyclical peak. <br><br>
* **Ex-Post Reality:** Even though it looked favorable, the trade lost money because the actual rise in rates (2022–2024) was much more aggressive than even the biased forward rates predicted. <br><br>
* **Context of Nov 2025:** Following the historic rate-hiking cycle, the yield curve in late 2025 is significantly **flatter**. Fact 3 implies that when the term spread is narrow, the expected excess return (the "carry") is lower. <br>

**Conclusion:** Because the curve is flatter in 2025 than in 2020, there is less "cushion" (lower term premium) to absorb further interest rate volatility. Consequently, the carry trade offers a **less attractive risk-adjusted return** for the 2025–2030 period than it did in 2020.